# LSTM Bot

## Project Overview

In this project, you will build a chatbot that can converse with you at the command line. The chatbot will use a Sequence to Sequence text generation architecture with an LSTM as it's memory unit. You will also learn to use pretrained word embeddings to improve the performance of the model. At the conclusion of the project, you will be able to show your chatbot to potential employers.

Additionally, you have the option to use pretrained word embeddings in your model. We have loaded Brown Embeddings from Gensim in the starter code below. You can compare the performance of your model with pre-trained embeddings against a model without the embeddings.



---



A sequence to sequence model (Seq2Seq) has two components:
- An Encoder consisting of an embedding layer and LSTM unit.
- A Decoder consisting of an embedding layer, LSTM unit, and linear output unit.

The Seq2Seq model works by accepting an input into the Encoder, passing the hidden state from the Encoder to the Decoder, which the Decoder uses to output a series of token predictions.

## Dependencies

- Pytorch
- Numpy
- Pandas
- NLTK
- Gzip
- Gensim


Please choose a dataset from the Torchtext website. We recommend looking at the Squad dataset first. Here is a link to the website where you can view your options:

- https://pytorch.org/text/stable/datasets.html





In [3]:
!pip install torchdata==0.3.0

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 47 kB 3.3 MB/s eta 0:00:011


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import gensim
import nltk
import tqdm

import numpy as np
import pandas as pd
import gzip
import time

from nltk.corpus import brown
# import dataset SQuAD2 as suggsted
from torchtext.datasets import SQuAD2

from torch.utils.data import DataLoader

nltk.download('brown')
nltk.download('punkt')

# Output, save, and load brown embeddings
model = gensim.models.Word2Vec(brown.sents())
model.save('brown.embedding')
w2v = gensim.models.Word2Vec.load('brown.embedding')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# import torchtext
# from torchtext.data import Field, BucketIterator

In [3]:
#You will use this function to load the dataset into a Pandas Dataframe for processing.
def loadDF(ds):
    df = {"question": [], "answer": []}
    for context, question, answers, indices in ds:
        if answers[0]:
            df["question"].append(question)
            df["answer"].append(answers[0])
    return pd.DataFrame.from_dict(df)
    

def prepare_text(sentence):
    '''
    Our text needs to be cleaned with a tokenizer. This function will perform that task.
    https://www.nltk.org/api/nltk.tokenize.html
    '''
    return nltk.tokenize.word_tokenize(sentence)


def train_test_split(SRC, TRG):
    '''
    Input: SRC, our list of questions from the dataset
           TRG, our list of responses from the dataset
    Output: Training and test datasets for SRC & TRG
    '''
    
    SRC_train_dataset = train_df["question"].tolist()
    TRG_train_dataset = train_df["answer"].tolist()
    
    SRC_test_dataset = test_df["question"].tolist()
    TRG_test_dataset = test_df["answer"].tolist()
        
    return SRC_train_dataset, SRC_test_dataset, TRG_train_dataset, TRG_test_dataset


In [4]:
train_dataset, test_dataset = SQuAD2()

In [5]:
train_df = loadDF(train_dataset)
test_df = loadDF(test_dataset)

In [6]:
SRC_train_dataset, SRC_test_dataset, TRG_train_dataset, TRG_test_dataset = train_test_split(train_df,test_df)

In [7]:
print(SRC_train_dataset[0])
print(TRG_train_dataset[0])

print(SRC_test_dataset[0])
print(TRG_test_dataset[0])

When did Beyonce start becoming popular?
in the late 1990s
In what country is Normandy located?
France


In [8]:
from torch.utils.data import Dataset

In [9]:
# create custom dataset class
class CustomTextDataset(Dataset):
    def __init__(self, questions, answers):
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.answers)

    def __getitem__(self, idx):
        answer = self.answers[idx]
        question = self.questions[idx]
        sample = {"Question": question, "Answer": answer}
        return sample


In [10]:
# define data and class labels
questions = SRC_train_dataset
answers = TRG_train_dataset

test_questions = SRC_test_dataset
test_answers = TRG_test_dataset

# create Pandas DataFrame
train_qa_df = pd.DataFrame({'Question': questions, 'Answer': answers})
test_qa_df = pd.DataFrame({'Question': test_questions, 'Answer': test_answers})

# define data set object
Train_TD = CustomTextDataset(train_qa_df['Question'], train_qa_df['Answer'])
Test_TD = CustomTextDataset(test_qa_df['Question'], test_qa_df['Answer'])

# Display image and label.
print('\nFirst iteration of data set: ', next(iter(Train_TD)), '\n')

print('\nFirst iteration of data set: ', next(iter(Test_TD)), '\n')

# Print how many items are in the data set
#print('Length of data set: ', len(TD), '\n')

# Print entire data set
#print('Entire data set: ', list(DataLoader(TD)), '\n')





First iteration of data set:  {'Question': 'When did Beyonce start becoming popular?', 'Answer': 'in the late 1990s'} 


First iteration of data set:  {'Question': 'In what country is Normandy located?', 'Answer': 'France'} 



In [71]:
train_iterator = DataLoader(Train_TD, batch_size=10, shuffle=True)
test_iterator = DataLoader(Test_TD, batch_size=10, shuffle=True)
# for (idx, batch) in enumerate(DL_DS):
#     # Print the 'text' data of the batch
#     print('Question data: ', batch['Question'])
#     # Print the 'class' data of batch
#     print('Answer data: ', batch['Answer'], '\n')

In [72]:
# use default configurations
#train_iter, test_iter = SQuAD2().iters(batch_size=128)
#train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=lambda x:x )
#test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True, collate_fn=lambda x:x )

In [73]:
# train_dataiter = next(iter(train_dataloader))
# test_dataiter = next(iter(test_dataloader))
# # print(train_dataiter.next())
# cont,q,a[0],idx[0] = train_dataiter.next()
# print(q)

# print(train_dataiter[0])
# print("/n")
# print(test_dataiter[0])

In [74]:
# for batch_idx, samples in enumerate(train_dataloader):
#       print(batch_idx, samples)

In [75]:
# num_epochs = 1
# for epoch in range(num_epochs):
#         for batch_idx, (x, y, z, a) in enumerate(train_dataloader):
#             if batch_idx >= 3:
#                 break
#             print(" Batch index:", batch_idx, end="")
#             print(" | Batch size:", y.shape[0], end="")
#             print(" | x shape:", x.shape, end="")
#             print(" | y shape:", y.shape)            
# print("Labels from current batch:", y)

In [76]:
import util as util
print("Start preparing training data ...")
voc, pairs = util.readVocs(train_df, "train")
print("Read {!s} sentence pairs".format(len(pairs)))

Start preparing training data ...
Reading lines...
Read 86821 sentence pairs


In [77]:
pairs = util.filterPairs(pairs)
print("Trimmed to {!s} sentence pairs".format(len(pairs)))

Trimmed to 29334 sentence pairs


In [78]:
print("Counting words...")
for pair in pairs:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("Counted words:", voc.num_words)

Counting words...
Counted words: 28298


In [79]:
for pair in pairs[:5]:
    print(pair)

['when did beyonce start becoming popular ?', 'in the late s']
['in which decade did beyonce become famous ?', 'late s']
['what album made her a worldwide known artist ?', 'dangerously in love']
['who managed the destiny s child group ?', 'mathew knowles']
['when did beyonce rise to fame ?', 'late s']


In [80]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 9536 / 28295 = 0.3370
Trimmed from 29334 pairs to 13802, 0.4705 of total


In [81]:
print(dir(voc))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'addSentence', 'addWord', 'index2word', 'name', 'num_words', 'trim', 'trimmed', 'word2count', 'word2index']


In [82]:
voc.num_words

9539

In [83]:
"""
Courtesy: http://ethen8181.github.io/machine-learning/deep_learning/seq2seq/1_torch_seq2seq_intro.html
    Input :
        - source batch
    Layer : 
        source batch -> Embedding -> LSTM
    Output :
        - LSTM hidden state
        - LSTM cell state
        
    Parmeters
    ---------
    input_dim : int
        Input dimension, should equal to the source vocab size.
    
    embedding_dim : int
        Embedding layer's dimension.
        
    hidden_dim : int
        LSTM Hidden/Cell state's dimension.
        
    *num_layers : int
        Number of LSTM layers.
        
    *dropout : float
        Dropout for the LSTM layer.

"""
class Encoder(nn.Module): 
    def __init__(self, input_dim, hidden_dim, embedding_dim):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.hidden = torch.zeros(1, 1, hidden_dim)
        # self.embedding provides a vector representation of the inputs to our model
        self.embedding = nn.Embedding(num_embeddings=self.input_dim, 
                                      embedding_dim=self.embedding_dim)
        
        # self.lstm, accepts the vectorized input and passes a hidden state
        self.lstm = nn.LSTM(input_size=self.embedding_dim,
                            hidden_size=self.hidden_dim,
                            num_layers=1)
    
    def forward(self, i):
        
        '''
        Inputs: i, the src vector
        Outputs: o, the encoder outputs
                h, the hidden state
                c, the cell state
        '''
        embedded = self.embedding(i)
        # outputs -> [sent len, batch size, hidden dim * n directions]
        #return o, h, c
        #o, (h, c) = self.lstm(embedded)
        return self.lstm(embedded)
    

In [84]:
"""
    Input :
        - first token in the target batch
        - LSTM hidden state from the encoder
        - LSTM cell state from the encoder
    Layer :
        target batch -> Embedding -- 
                                   |
        encoder hidden state ------|--> LSTM -> Linear
                                   |
        encoder cell state   -------
        
    Output :
        - prediction
        - LSTM hidden state
        - LSTM cell state

    Parmeters
    ---------
    output : int
        Output dimension, should equal to the target vocab size.
    
    emb_dim : int
        Embedding layer's dimension.
        
    hid_dim : int
        LSTM Hidden/Cell state's dimension.
        
    *num_layers : int
        Number of LSTM layers.
        
    dropout : float
        Dropout for the LSTM layer.
    """
class Decoder(nn.Module):
    def __init__(self, hidden_dim, output, embedding_dim):
        
        super(Decoder, self).__init__()
        self.hidden_dim=hidden_dim
        self.output=output
        self.embedding_dim=embedding_dim
        
        # self.embedding provides a vector representation of the target to our model
        self.embedding = nn.Embedding(num_embeddings=self.output,
                                      embedding_dim=self.embedding_dim)
       
        # self.lstm, accepts the embeddings and outputs a hidden state
        self.lstm = nn.LSTM(self.embedding_dim, hidden_dim, num_layers=3)
        
        # self.ouput, predicts on the hidden state via a linear output layer     
        self.out = nn.Linear(self.hidden_dim, self.output)
        
    def forward(self, i, h, c):
        
        '''
        Inputs: i, the target vector
        Outputs: o, the prediction
                h, the hidden state
        '''
        unsqueezed = i.unsqueeze(0)
        embedded = self.embedding(unsqueezed)
        
        o, h, c = self.lstm(embedded, (h, c))
        
        p = self.out(o.squeeze(0))
        
        return o, h, p, c

In [85]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
        assert encoder.hidden_dim == decoder.hidden_dim, \
            'Hidden dimensions of encoder and decoder must be equal!'
#         assert encoder.n_layers == decoder.n_layers, \
#             'Encoder and decoder must have equal number of layers!'
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        #batch_size = trg.shape[1]
        batch_size = len(trg)
        trg_len = len(trg)
        trg_vocab_size = self.decoder.output
        
        o = torch.zeros(trg_len, batch_size, trg_vocab_size)
#       train_df["label"] = train_df["label"].apply(lambda x: list(map(int, x)))
        h, c = self.encoder(torch.tensor(src))
        
        x = trg[0, :]
        for t in range(1, trg_len):
            o, h, c = self.decoder(x, h, c)
            o[t] = o
            teacher_force = random.random() < teacher_forcing_ratio
            top_trg = o.argmax(1)
            x = trg[t] if teacher_force else top_trg
            
        return o
    
    
#     def forward(self, src_batch: torch.LongTensor, trg_batch: torch.LongTensor,
#                 teacher_forcing_ratio: float=0.5):

#         max_len, batch_size = trg_batch.shape
#         trg_vocab_size = self.decoder.output_dim

#         # tensor to store decoder's output
#         outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

#         # last hidden & cell state of the encoder is used as the decoder's initial hidden state
#         hidden, cell = self.encoder(src_batch)

#         trg = trg_batch[0]
#         for i in range(1, max_len):
#             prediction, hidden, cell = self.decoder(trg, hidden, cell)
#             outputs[i] = prediction

#             if random.random() < teacher_forcing_ratio:
#                 trg = trg_batch[i]
#             else:
#                 trg = prediction.argmax(1)

#         return outputs

In [86]:
# First initialize our model.
input_size = voc.num_words
output_size = voc.num_words
embedding_size = 256
hidden_size = 512

In [87]:
encoder = Encoder(input_dim=input_size,   hidden_dim=hidden_size, embedding_dim=embedding_size)
decoder = Decoder(hidden_dim=hidden_size, output=output_size, embedding_dim=embedding_size)

model = Seq2Seq(encoder, decoder)

In [88]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9539, 256)
    (lstm): LSTM(256, 512)
  )
  (decoder): Decoder(
    (embedding): Embedding(9539, 256)
    (lstm): LSTM(256, 512, num_layers=3)
    (out): Linear(in_features=512, out_features=9539, bias=True)
  )
)

In [89]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,133,891 trainable parameters


## Training the model

In [90]:
# Defining hyperparameters
lr = 0.001

In [91]:
#Selecting the optimizer and criterion
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [92]:
# SRC_train_dataset[0] 
#BUCKET ITERATOR
#pairs[10]

In [93]:
def train(model, iterator, optimizer, criterion):
    model.train()

    epoch_loss = 0
    for idx,batch in enumerate(iterator):
        optimizer.zero_grad()
        outputs = model(batch['Question'], batch['Answer'])

        # 1. as mentioned in the seq2seq section, we will
        # cut off the first element when performing the evaluation
        # 2. the loss function only works on 2d inputs
        # with 1d targets we need to flatten each of them
        outputs_flatten = outputs[1:].view(-1, outputs.shape[-1])
#       trg_flatten = batch.trg[1:].view(-1)
        trg_flatten = batch['Answer'][1:].view(-1)
        loss = criterion(outputs_flatten, trg_flatten)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [94]:
# test_batch = iter(test_dataloader)
# testbatch = test_batch.next()
# testbatch.src

# train_dataloader = next(iter(train_dataloader))

In [95]:
# for idx,batch in enumerate(train_dataloader):
#         print(batch[idx][2])

In [96]:
def evaluate(model, iterator, criterion):
    model.eval()

    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            # turn off teacher forcing
            outputs = model(batch['Question'], batch['Answer'], teacher_forcing_ratio=0) 

            # trg = [trg sent len, batch size]
            # output = [trg sent len, batch size, output dim]
            outputs_flatten = outputs[1:].view(-1, outputs.shape[-1])
            trg_flatten = batch.trg[1:].view(-1)
            loss = criterion(outputs_flatten, trg_flatten)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [97]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [98]:
N_EPOCHS = 20
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):    
    start_time = time.time()
    train_loss = train(model, train_iterator, optimizer, criterion)
    valid_loss = evaluate(model, test_iterator, criterion)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'chatbotmodel.pt')

    # it's easier to see a change in perplexity between epoch as it's an exponential
    # of the loss, hence the scale of the measure is much bigger
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

ValueError: too many dimensions 'str'

## Evaluating Seq2Seq

In [ ]:
seq2seq.load_state_dict(torch.load('chatbotmodel.pt'))

test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
example_idx = 0
example = train_data.examples[example_idx]
print('source sentence: ', ' '.join(example.src))
print('target sentence: ', ' '.join(example.trg))